In [14]:
%load_ext watermark
%watermark

%load_ext autoreload
%autoreload 2


# import standard libs
from IPython.display import display
from IPython.core.debugger import set_trace as bp
from pathlib import PurePath, Path
import sys
import time
from collections import OrderedDict as od
import re
import os
import json
import datetime
import pickle


# import python scientific stack
import pandas as pd
import pandas_datareader.data as web
pd.set_option('display.max_rows', 10)
from dask import dataframe as dd
from dask.diagnostics import ProgressBar
from multiprocessing import cpu_count
pbar = ProgressBar()
pbar.register()
import numpy as np
import scipy.stats as stats
import statsmodels.api as sm
from numba import jit
import math
# import ffn


# import visual tools
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline
import seaborn as sns

plt.style.use('seaborn-talk')
plt.style.use('bmh')
#plt.rcParams['font.family'] = 'DejaVu Sans Mono'
plt.rcParams['font.size'] = 9.5
plt.rcParams['font.weight'] = 'medium'
plt.rcParams['figure.figsize'] = 10,7
blue, green, red, purple, gold, teal = sns.color_palette('colorblind', 6)

RANDOM_STATE = 777

print()

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
Last updated: 2024-09-18T14:45:52.590409-04:00

Python implementation: CPython
Python version       : 3.8.19
IPython version      : 8.12.2

Compiler    : Clang 16.0.6 
OS          : Darwin
Release     : 23.6.0
Machine     : arm64
Processor   : arm
CPU cores   : 8
Architecture: 64bit

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload



/var/folders/j0/8v9qcjfx15g5ftmsy5n0qrq80000gn/T/ipykernel_40702/2173331131.py:46: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-talk')


In [15]:
import os

# Run the setup script
%run ../../config/setup_project.py

# Call the function to set up the project path
setup_project_path()

# Now you can import your modules
from src.utils import helper as h_
import src.ch_02.code_ch_02 as f_ch2
import src.ch_03.code_ch_03 as f_ch3

Project root added to sys.path: /Users/paulkelendji/Desktop/GitHub_paul/ML-Asset_Management
Config path added to sys.path: /Users/paulkelendji/Desktop/GitHub_paul/ML-Asset_Management/config
Current sys.path: ['/Users/paulkelendji/miniconda3/envs/financial_math/lib/python38.zip', '/Users/paulkelendji/miniconda3/envs/financial_math/lib/python3.8', '/Users/paulkelendji/miniconda3/envs/financial_math/lib/python3.8/lib-dynload', '', '/Users/paulkelendji/miniconda3/envs/financial_math/lib/python3.8/site-packages', '/Users/paulkelendji/miniconda3/envs/financial_math/lib/python3.8/site-packages/setuptools/_vendor', '/Users/paulkelendji/Desktop/GitHub_paul/ML-Asset_Management', '/Users/paulkelendji/Desktop/GitHub_paul/ML-Asset_Management/config', '/Users/paulkelendji/Desktop/GitHub_paul/ML-Asset_Management', '/Users/paulkelendji/Desktop/GitHub_paul/ML-Asset_Management', '/Users/paulkelendji/Desktop/GitHub_paul/ML-Asset_Management', '/Users/paulkelendji/Desktop/GitHub_paul/ML-Asset_Management/co

In [16]:
df = pd.read_parquet("../../data/IVE_kibot.parq")
df

FileNotFoundError: [Errno 2] No such file or directory: '../../data/IVE_kibot.parq'

### **SNIPPET 4.1** **ESTIMATING THE UNIQUENESS OF A LABEL**


In [17]:
# def mpNumCoEvents(closeIdx, t1, molecule):
#     '''
#     Compute the number of concurrent events per bar.
#     +molecule[0] is the date of the first event on which the weight will be computed
#     +molecule[-1] is the date of the last event on which the weight will be computed
#     Any event that starts before t1[molecule].max() impacts the count.
#     '''
#     #1) find events that span the period [molecule[0], molecule[-1]]
#     t1 = t1.fillna(closeIdx[-1])  # unclosed events still must impact other weights
#     t1 = t1[t1 >= molecule[0]]  # events that end at or after molecule[0]
#     t1 = t1.loc[:t1[molecule].max()]  # events that start at or before t1[molecule].max()
    
#     #2) count events spanning a bar
#     iloc = closeIdx.searchsorted(np.array([t1.index[0], t1.max()]))
#     count = pd.Series(0, index=closeIdx[iloc[0]:iloc[1] + 1])
#     for tIn, tOut in t1.iteritems():
#         count.loc[tIn:tOut] += 1
#     return count.loc[molecule[0]:t1[molecule].max()]


The equation \( I^{-1} \sum_{i=1}^{I} \tilde{u}_i \ll 1 \) refers to the average uniqueness of the labels in the context of bagging classifiers, particularly when using the bootstrap method in machine learning. 

Here's what this equation represents:

- **\( \tilde{u}_i \)**: This term represents the average uniqueness of label \( i \). Uniqueness, in this context, refers to how distinct or non-overlapping a particular label is over its lifespan in the dataset.

- **\( I \)**: This is the total number of labels.

- **\( I^{-1} \sum_{i=1}^{I} \tilde{u}_i \)**: This is the average of the uniqueness values across all labels. It tells us how unique, on average, the labels are in the dataset.

- **\( \ll 1 \)**: This symbol indicates that the average uniqueness is much less than 1. In other words, it suggests that the labels are highly overlapping or redundant.

### Interpretation in Context:

The text is discussing the issue of redundancy when sampling with replacement (as in the bootstrap method) in the presence of highly overlapping labels (low uniqueness). If the average uniqueness \( I^{-1} \sum_{i=1}^{I} \tilde{u}_i \) is very small (i.e., much less than 1), this implies that most of the labels are overlapping significantly. 

This has a couple of implications:

1. **Redundant Samples**: If uniqueness is low, it means that different bootstrap samples are likely to contain very similar or even identical information. This reduces the effectiveness of bagging, as the different models trained on these bootstrap samples will not be as diverse as they should be.

2. **Inefficient Bootstrapping**: The bootstrapping process becomes inefficient because it doesn't capture enough variability in the data. This can lead to overfitting, as the models trained on these bootstrap samples become too similar to each other, essentially learning the same patterns and making the same errors.

The equation thus highlights the importance of ensuring that the labels or observations used in the bootstrap samples are sufficiently unique or non-overlapping to make the bagging process effective. If the average uniqueness is too low, the benefits of bagging (like reduced variance and improved generalization) may not be fully realized.

# Framework

In [18]:
# load ../data/variables_ch2.pkl
%run ../ch_02/code_ch_02.py

path = '../../data/variables_ch2.pkl'
import pickle
with open(path, 'rb') as f:
    bars = pickle.load(f)
    bar_time = pickle.load(f)
    
# df as bars['Dollar'].df_OLHC without 'cusum' column
df = bars['Dollar'].df_OLHC.drop(columns=['cusum'])
# For the purpose of this example, remove rows where time_close is duplicated
# (keep the first row)
# Remove rows where time_close is duplicated, keeping the first occurrence
df = df.drop_duplicates(subset='time_close', keep='first')
# set index as 'time_close'
df = df.set_index('time_close')
df

,time_open,open,low,high,close,vwap,volume
time_close,,,,,,,
2009-09-29 13:23:42,2009-09-28 09:30:00,50.7900,50.710,51.9600,51.730,51.430024,701459
2009-10-01 13:44:06,2009-09-29 13:26:31,51.7275,50.070,51.7600,50.270,50.908633,709392
2009-10-02 14:37:25,2009-10-01 13:45:10,50.2376,49.190,50.3166,49.761,49.836850,724111
2009-10-07 11:39:00,2009-10-02 14:38:03,49.7635,49.440,51.4700,51.090,50.606475,713127
2009-10-12 14:24:38,2009-10-07 11:40:15,51.0880,50.980,52.2252,51.990,51.675670,698754
...,...,...,...,...,...,...,...
2024-07-25 11:50:36,2024-07-24 15:51:05,186.6706,186.460,188.5800,188.400,187.387274,186271
2024-07-25 14:46:14,2024-07-25 11:50:40,188.4000,187.685,189.1286,187.685,188.380972,194394
2024-07-26 10:11:15,2024-07-25 14:46:35,187.6150,186.980,189.2300,189.020,187.749127,195797


In [19]:
# Step 1 : get the daily volatility
close = df['close']
dailyVol = f_ch3.getDailyVol(close, span0=100).dropna()

# Step 2 : Detect events (significant points in time where price changes)
yt = close.pct_change().dropna()
tEvents = f_ch3.getTEvents(yt, h=dailyVol)

# Step 3 : Obtain t1 series
NUM_DAYS = 10
t1 = f_ch3.addVerticalBarrier(tEvents, close, numDays=NUM_DAYS)

# Step 4 : Filter events where target is higher than threshold
# Create target series
ptsl = [1,1]
target=dailyVol
# Select minRet
minRet = 0.01
events = f_ch3.getEvents(close,tEvents,ptsl,target,minRet,numThreads = 1 ,t1=t1)

In [20]:
events

,t1,trgt
2009-10-02 14:37:25,2009-10-07 11:39:00,0.012797
2009-10-07 11:39:00,2009-10-15 12:03:22,0.028053
2009-10-20 11:12:37,2009-10-27 11:06:15,0.020279
2009-10-27 11:06:15,2009-11-03 11:21:01,0.020823
2009-11-02 10:30:21,2009-11-10 10:55:00,0.018378
...,...,...
2023-11-02 13:19:47,2023-11-03 12:07:30,0.010664
2023-11-03 12:07:30,2023-11-14 10:00:23,0.011264
2023-11-03 15:23:46,2023-11-14 10:00:23,0.011404
2023-11-14 10:00:23,2023-11-20 13:08:22,0.011173


In [21]:
t1

2009-10-02 14:37:25   2009-10-15 12:03:22
2009-10-07 11:39:00   2009-10-20 11:12:37
2009-10-20 11:12:37   2009-11-02 10:30:21
2009-10-27 11:06:15   2009-11-10 10:55:00
2009-11-02 10:30:21   2009-11-13 12:12:50
                              ...        
2024-07-02 14:11:15   2024-07-12 16:00:00
2024-07-10 13:21:10   2024-07-22 14:12:45
2024-07-11 12:01:59   2024-07-22 14:12:45
2024-07-12 16:00:00   2024-07-23 10:51:52
2024-07-16 10:07:09   2024-07-26 10:11:15
Name: time_close, Length: 1513, dtype: datetime64[ns]

In [22]:
close

time_close
2009-09-29 13:23:42     51.730
2009-10-01 13:44:06     50.270
2009-10-02 14:37:25     49.761
2009-10-07 11:39:00     51.090
2009-10-12 14:24:38     51.990
                        ...   
2024-07-25 11:50:36    188.400
2024-07-25 14:46:14    187.685
2024-07-26 10:11:15    189.020
2024-07-26 15:59:52    189.460
2024-07-26 16:00:00    189.390
Name: close, Length: 4231, dtype: float64

In [23]:
# **SNIPPET 4.1** **ESTIMATING THE UNIQUENESS OF A LABEL**

def mpNumCoEvents(closeIdx, t1, molecule):
    '''
    Compute the number of concurrent events per bar.
    +molecule[0] is the date of the first event on which the weight will be computed
    +molecule[-1] is the date of the last event on which the weight will be computed
    Any event that starts before t1[molecule].max() impacts the count.
    '''
    #1) find events that span the period [molecule[0], molecule[-1]]
    t1 = t1.fillna(closeIdx[-1])  # unclosed events still must impact other weights
    t1 = t1[t1 >= molecule[0]]  # events that end at or after molecule[0]
    t1 = t1.loc[:t1[molecule].max()]  # events that start at or before t1[molecule].max()
    
    #2) count events spanning a bar
    iloc = closeIdx.searchsorted(np.array([t1.index[0], t1.max()]))
    count = pd.Series(0, index=closeIdx[iloc[0]:iloc[1] + 1])
    for tIn, tOut in t1.items():  # Changed from iteritems() to items()
        count.loc[tIn:tOut] += 1
    return count.loc[molecule[0]:t1[molecule].max()]

In [24]:
# # Step 1: Compute the number of concurrent events
# numCoEvents = mpNumCoEvents(close.index, events.t1, events.index)

In [25]:
out = f_ch3.getBins(events, close)
out

,ret,bin
2009-10-02 14:37:25,0.026708,1.0
2009-10-07 11:39:00,0.033647,1.0
2009-10-20 11:12:37,-0.034529,-1.0
2009-10-27 11:06:15,-0.028689,-1.0
2009-11-02 10:30:21,0.038788,1.0
...,...,...
2023-11-02 13:19:47,0.016538,1.0
2023-11-03 12:07:30,0.023062,1.0
2023-11-03 15:23:46,0.021314,1.0
2023-11-14 10:00:23,0.015028,1.0


In [26]:
# **SNIPPET 4.2** **ESTIMATING THE AVERAGE UNIQUENESS OF A LABEL**

def mpSampleTW(t1, numCoEvents, molecule):
    # Derive average uniqueness over the event's lifespan
    wght = pd.Series(index=molecule)
    for tIn, tOut in t1.loc[wght.index].items():  # Changed from .iteritems() to .items()
        wght.loc[tIn] = (1. / numCoEvents.loc[tIn:tOut]).mean()
    return wght

In [27]:
numCoEvents = f_ch3.mpPandasObj(
    mpNumCoEvents,
    ("molecule", events.index),
    numThreads=1,
    closeIdx=close.index,
    t1=events["t1"],
)

numCoEvents = numCoEvents.loc[~numCoEvents.index.duplicated(keep="last")]
numCoEvents=numCoEvents.reindex(close.index).ﬁllna(0)

out["tW"] = f_ch3.mpPandasObj(
    mpSampleTW,
    ("molecule", events.index),
    numThreads=1,
    t1=events["t1"],
    numCoEvents=numCoEvents,
)

# average uniqueness of a label
print(f"The average uniqueness of a label is: {out['tW'].mean()}")

The average uniqueness of a label is: 0.5106862013855774


In [28]:
out

,ret,bin,tW
2009-10-02 14:37:25,0.026708,1.0,0.750000
2009-10-07 11:39:00,0.033647,1.0,0.833333
2009-10-20 11:12:37,-0.034529,-1.0,0.833333
2009-10-27 11:06:15,-0.028689,-1.0,0.625000
2009-11-02 10:30:21,0.038788,1.0,0.500000
...,...,...,...
2023-11-02 13:19:47,0.016538,1.0,0.444444
2023-11-03 12:07:30,0.023062,1.0,0.452381
2023-11-03 15:23:46,0.021314,1.0,0.472222
2023-11-14 10:00:23,0.015028,1.0,0.476190


In [29]:
# SNIPPET 4.3 BUILD AN INDICATOR MATRIX
def getIndMatrix(barIx, t1):
    # Get indicator matrix
    indM = pd.DataFrame(0, index=barIx, columns=range(t1.shape[0]))
    for i, (t0, t1) in enumerate(t1.items()):  # Changed from .iteritems() to .items()
        indM.loc[t0:t1, i] = 1.
    return indM

# SNIPPET 4.4 COMPUTE AVERAGE UNIQUENESS
def getAvgUniqueness(indM):
    # Average uniqueness from indicator matrix
    c = indM.sum(axis=1)  # concurrency
    u = indM.div(c, axis=0)  # uniqueness
    avgU = u[u > 0].mean()  # average uniqueness
    return avgU

# SNIPPET 4.5 RETURN SAMPLE FROM SEQUENTIAL BOOTSTRAP
def seqBootstrap(indM, me=None):
    # Generate a sample via sequential bootstrap
    if sLength is None:
        sLength = indM.shape[1]
    phi = []
    while len(phi) < sLength:
        avgU = pd.Series()
        for i in indM:
            indM_ = indM[phi + [i]]  # reduce indM
            avgU.loc[i] = getAvgUniqueness(indM_).iloc[-1]
        prob = avgU / avgU.sum()  # draw prob
        phi += [np.random.choice(indM.columns, p=prob)]
    return phi

from tqdm import tqdm
# SNIPPET 4.5 RETURN SAMPLE FROM SEQUENTIAL BOOTSTRAP
def seqBootstrap(indM, sLength=None, uniqueObj = 0.8):
    # Generate a sample via sequential bootstrap
    if sLength is None:
        sLength = indM.shape[1]
    phi = []
    
    # Use tqdm to add a progress bar
    for _ in tqdm(range(sLength), desc="Sequential Bootstrap Progress"):
        avgU = pd.Series()
        for i in indM:
            indM_ = indM[phi + [i]]  # reduce indM
            avgU.loc[i] = getAvgUniqueness(indM_).iloc[-1]
        prob = avgU / avgU.sum()  # draw prob
        phi += [np.random.choice(indM.columns, p=prob)]
        if len(phi) % 50 == 0:
            print(f"Sample length: {len(phi)}")
            print('Sequential uniqueness:',getAvgUniqueness(indM[phi]).mean())
            print()
        if getAvgUniqueness(indM[phi]).mean() < uniqueObj:
            break
    
    return phi


In [30]:
# SNIPPET 4.6 EXAMPLE OF SEQUENTIAL BOOTSTRAP

def main(t1, **kwargs):
    # t1=pd.Series([2,3,5],index=[0,2,4]) # t0,t1 for each feature obs
    # barIx=range(t1.max()+1) # index of bars
    barIx = t1.index
    indM=getIndMatrix(barIx,t1)
    # phi=np.random.choice(indM.columns,size=indM.shape[1])
    phi=np.random.choice(indM.columns,size=kwargs['sLength'])
    # print(phi)
    print('Standard uniqueness:',getAvgUniqueness(indM[phi]).mean())
    phi=seqBootstrap(indM, **kwargs)
    # print(phi)
    print('Sequential uniqueness:',getAvgUniqueness(indM[phi]).mean())
    return phi

phi = main(t1=events['t1'], sLength=250, uniqueObj=0.5)

Standard uniqueness: 0.7425063492063493


Sequential Bootstrap Progress:  20%|██        | 50/250 [01:43<06:59,  2.10s/it]

Sample length: 50
Sequential uniqueness: 0.97



Sequential Bootstrap Progress:  40%|████      | 100/250 [04:13<11:09,  4.46s/it]

Sample length: 100
Sequential uniqueness: 0.9397222222222222



...

Sample length: 150
Sequential uniqueness: 0.9045370370370369



Sequential Bootstrap Progress:  80%|████████  | 200/250 [10:34<03:29,  4.20s/it]

Sample length: 200
Sequential uniqueness: 0.8707777777777779



Sequential Bootstrap Progress: 100%|██████████| 250/250 [15:27<00:00,  3.71s/it]

Sample length: 250
Sequential uniqueness: 0.8252888888888888

Sequential uniqueness: 0.8252888888888888


In [53]:
# events

In [54]:
out['t1'] = events['t1']
# out

In [55]:
# samples = out.iloc[phi]
# samples

In [46]:
# **SNIPPET 4.10** **DETERMINATION OF SAMPLE WEIGHT BY ABSOLUTE RETURN ATTRIBUTION**
def mpSampleW(t1, numCoEvents, close, molecule):
    # Derive sample weight by return attribution
    ret = np.log(close).diff()  # log-returns, so that they are additive
    wght = pd.Series(index=molecule)
    for tIn, tOut in t1.loc[wght.index].items():  # Changed from iteritems() to items()
        wght.loc[tIn] = (ret.loc[tIn:tOut] / numCoEvents.loc[tIn:tOut]).sum()
    return wght.abs()

In [48]:
out["w"] = f_ch3.mpPandasObj(
    mpSampleW,
    ("molecule", events.index),
    numThreads=1,
    t1=events["t1"],
    numCoEvents=numCoEvents,
    close=close,
)
out['w'] *= out.shape[0] / out['w'].sum()

In [ ]:
out

In [ ]:
# **SNIPPET 4.11** **IMPLEMENTATION OF TIME-DECAY FACTORS**
def getTimeDecay(tW, clfLastW=1.0):
    # apply piecewise-linear decay to observed uniqueness (tW)
    # newest observation gets weight=1, oldest observation gets weight=clfLastW
    clfW = tW.sort_index().cumsum()
    if clfLastW >= 0:
        slope = (1.0 - clfLastW) / clfW.iloc[-1]
    else:
        slope = 1.0 / ((clfLastW + 1) * clfW.iloc[-1])
    const = 1.0 - slope * clfW.iloc[-1]
    clfW = const + slope * clfW
    clfW[clfW < 0] = 0
    print(const, slope)
    return clfW

out["TdW"]=getTimeDecay(out["tW"], clfLastW=0.8)

In [57]:
out["W"]=out["w"]*out["TdW"]

In [ ]:
samples = out.iloc[phi]
samples